In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
### hack tf-keras to appear as top level keras
import sys
sys.modules['keras'] = keras
### end of hack
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
# from keras.utils import layer_utils
# from keras.utils.data_utils import get_file
# from keras.applications.imagenet_utils import preprocess_input
# import pydot
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import math
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from tensorflow.python.framework import ops


In [6]:
from random import randint
from sklearn.model_selection import train_test_split

In [19]:


train_count = 5000
test_count = 10
max_sent_length = 100
embedding_size = 300

X = np.random.randn(train_count, max_sent_length, embedding_size)
Y = np.zeros((train_count, 1))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

for i in range(x_train.shape[0]):
    y_train[i:] = randint(0, 5)
for i in range(x_test.shape[0]):
    y_test[i:] = randint(0, 5)


In [7]:
train_count = 50
test_count = 10
max_sent_length = 100
embedding_size = 300
label_head = [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
]
label_dim = len(label_head[0])
X = np.random.randn(train_count, max_sent_length, embedding_size)
Y = np.zeros((train_count, label_dim))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

for i in range(x_train.shape[0]):
    y_train[i:] = label_head[randint(0, (label_dim - 1))]
for i in range(x_test.shape[0]):
    y_test[i:] = label_head[randint(0, (label_dim - 1))]

In [8]:
def getModel(input_shape):
    X_input = keras.layers.Input(shape=input_shape)
    X = keras.layers.Lambda(lambda x: K.mean(x, axis=1, keepdims=True))(X_input)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(128, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(256, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(128, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(label_dim, activation=tf.nn.sigmoid)(X)
    model = keras.Model(inputs=X_input, outputs=X, name='HappyModel')
    return model


In [9]:
model = getModel((max_sent_length, embedding_size))
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 300)          0         
_________________________________________________________________
lambda (Lambda)              (None, 1, 300)            0         
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total para

In [11]:
model.fit(x=x_train, y=y_train, epochs=30)

Epoch 1/30
40/40 [==============================] - 1s 20ms/step - loss: 0.6885 - acc: 0.6000
Epoch 2/30
40/40 [==============================] - 0s 366us/step - loss: 0.6562 - acc: 0.8100
Epoch 3/30
40/40 [==============================] - 0s 496us/step - loss: 0.6270 - acc: 0.8000
Epoch 4/30
40/40 [==============================] - 0s 391us/step - loss: 0.5958 - acc: 0.8000
Epoch 5/30
40/40 [==============================] - 0s 306us/step - loss: 0.5616 - acc: 0.8000
Epoch 6/30
40/40 [==============================] - 0s 353us/step - loss: 0.5269 - acc: 0.8000
Epoch 7/30
40/40 [==============================] - 0s 351us/step - loss: 0.4931 - acc: 0.8000
Epoch 8/30
40/40 [==============================] - 0s 377us/step - loss: 0.4635 - acc: 0.8000
Epoch 9/30
40/40 [==============================] - 0s 384us/step - loss: 0.4429 - acc: 0.8000
Epoch 10/30
40/40 [==============================] - 0s 393us/step - loss: 0.4315 - acc: 0.8000
Epoch 11/30
40/40 [==============================]

In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Loss = %.2f  and test accuracy = %.2f' % (test_loss, test_acc))

10/10 [==============================] - 0s 6ms/step
Loss = 0.68  and test accuracy = 0.78


In [13]:
model.predict(x_test)

array([[0.00803457, 0.29191414, 0.06826521, 0.03567823, 0.10152549],
       [0.03211179, 0.4531743 , 0.10631808, 0.0773826 , 0.09524003],
       [0.6975581 , 0.02673708, 0.07784151, 0.02630389, 0.01813409],
       [0.03265885, 0.15210047, 0.03571992, 0.19157644, 0.02535217],
       [0.06136189, 0.21347149, 0.14523083, 0.11045967, 0.02498882],
       [0.15205604, 0.3110408 , 0.26016113, 0.06849097, 0.0465335 ],
       [0.09258208, 0.37675843, 0.14761288, 0.05586514, 0.00641896],
       [0.04616815, 0.16396703, 0.10943025, 0.04748256, 0.2175592 ],
       [0.44055447, 0.09731206, 0.13194409, 0.0600808 , 0.04749652],
       [0.12554006, 0.04594212, 0.19889203, 0.19073395, 0.20170808]],
      dtype=float32)

In [16]:
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output = get_3rd_layer_output([x_test[:1]])[0]

In [17]:
layer_output.shape

(1, 128)